# Leaf variables

In [1]:
from IPython import get_ipython
if get_ipython():
    get_ipython().run_line_magic('load_ext', 'autoreload')
    get_ipython().run_line_magic('autoreload', '2')

import latenta as la
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import torch

In [2]:
cells = la.Dim(10000, "cell")
genes = la.Dim(10000, "gene")

## Loaders

In [3]:
value = np.zeros((len(cells), len(genes)))
for i in range(value.shape[0]):
    value[i, np.random.choice(value.shape[1])] = 1.

In [4]:
value.sum()

10000.0

### Memory loader

In [7]:
loader = la.variables.loaders.MemoryLoader(value)
fixed = la.Fixed(loader, definition = la.Definition([cells, genes]))

### Sparse loader

In [8]:
import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType

def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
            if torch.is_tensor(obj):
                size += obj.element_size() * obj.nelement()
        objects = get_referents(*need_referents)
    return size

Obviously, a large difference in memory usage.

In [9]:
getsize(loader)/1024/1024

381.47050285339355

Using 3GB (or more) on current GPUs would be prohibitive.

In [10]:
loader_sparse = la.variables.loaders.SparseDenseMemoryLoader(la.sparse.COO.from_numpy_array(value))
fixed_sparse = la.Fixed(loader_sparse, definition = la.Definition([cells, genes]))

In [11]:
getsize(loader_sparse)/1024/1024

1.5413875579833984

#### Initialization

There are several ways to initialize the COO:

In [12]:
la.sparse.COO.from_numpy_array(value)

COO(row=tensor([   0,    1,    2,  ..., 9997, 9998, 9999]), col=tensor([4780, 4479, 1517,  ..., 3844, 9554, 7926]), values=tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64), shape=(10000, 10000), mapping=None)

In [13]:
value_scipy_coo = scipy.sparse.coo_matrix(value)
la.sparse.COO.from_scipy_coo(value_scipy_coo)

COO(row=tensor([   0,    1,    2,  ..., 9997, 9998, 9999]), col=tensor([4780, 4479, 1517,  ..., 3844, 9554, 7926]), values=tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64), shape=(10000, 10000), mapping=None)

In [14]:
value_scipy_csr = scipy.sparse.csr_matrix(value)
la.sparse.COO.from_scipy_csr(value_scipy_csr)

COO(row=tensor([   0,    1,    2,  ..., 9997, 9998, 9999]), col=tensor([4780, 4479, 1517,  ..., 3844, 9554, 7926]), values=tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64), shape=(10000, 10000), mapping=None)

#### Timing

When data is accessed from a SparseDenseMemoryLoader, it is converted to a dense tensor. This can cause a slowdown:

In [15]:
import timeit

In [16]:
time = timeit.timeit("loader.get()", number = 1, globals = globals())
time

3.4299446269869804e-06

In [17]:
time = timeit.timeit("loader_sparse.get()", number = 1, globals = globals())
time

0.05968685192056

Speed is however more competitive when taking only a subsample:

In [18]:
idx = {"cell":torch.tensor(range(500))}

In [19]:
time = timeit.timeit("loader.get(idx)", number = 1, globals = globals())
time

0.00203545403201133

In [20]:
time = timeit.timeit("loader_sparse.get(idx)", number = 1, globals = globals())
time

0.003830755944363773